<a href="https://colab.research.google.com/github/codeREXus/langchain-learnings/blob/main/Langchain5_chromaDB_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install chromadb==1.0.12
!pip install sentence-transformers==4.1.0

In [2]:
import chromadb
from chromadb.utils import embedding_functions

ef= embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name ="all-MiniLm-L6-V2"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
client=chromadb.Client()
collection_name= 'my_grocery'

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


In [18]:
def main():
  try:
    global collection
    collection = client.create_collection(
    name=collection_name,
    metadata={"description": "A collection for storing grocery data"},
    configuration={
        "hnsw": {"space": "cosine"},
        "embedding_function": ef,
        'ef_search':60,
        'ef_construction':120
        }
    )
    print(f"Collection created: {collection.name}")


    texts = [
    'fresh red apples',
    'organic bananas',
    'ripe mangoes',
    'whole wheat bread',
    'farm-fresh eggs',
    'natural yogurt',
    'frozen vegetables',
    'grass-fed beef',
    'free-range chicken',
    'fresh salmon fillet',
    'aromatic coffee beans',
    'pure honey',
    'golden apple',
    'red fruit'
    ]


    ids = [f"food_{index + 1}" for index, _ in enumerate(texts)]

    collection.add(
    documents=texts,
    metadatas=[{'source':'grocery', 'catgory':'food'} for _ in texts],
    ids= ids
    )
    all_item= collection.get()


    sm_search(collection, all_item)
  except Exception as e:
    print(f"Error {e}")

In [22]:
def sm_search(collection, all_item):
  try:
    global query_term
    query_term=[ 'carrot','potato']
    if isinstance(query_term, str):
        query_term = [query_term]
    else:
        raise ValueError("query_term must be a string or a list of strings")

    results = collection.query(
    query_texts=[query_term],
    n_results=3
    )
    print(f"Query results for '{query_term}':")
    print(results)

    if not results or not results['ids'] or len(results['ids'][0]) == 0:
      print(f'No documents found similar to "{query_term}"')
      return

    print(f'Top 3 similar documents to "{query_term}":')

    for i in range(min(3, len(results['ids'][0]))):
        doc_id = results['ids'][0][i]
        score = results['distances'][0][i]  # Get score from 'distances' array
        text = results['documents'][0][i]
        if not text:
            print(f' - ID: {doc_id}, Text: "Text not available", Score: {score:.4f}')
        else:
            print(f' - ID: {doc_id}, Text: "{text}", Score: {score:.4f}')
  except Exception as error :
    print(f"Error in similarity search: {error}")
  except Exception as e:
    print(f"Error {e}")